In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
import plotly.graph_objects as go

file_path = '../data/Банки.xlsx'
data = pd.read_excel(file_path, sheet_name='Лист1')

# Шаг 1: Упорядочить критерии по важности
# Пример: средняя ставка по кредиту, минимальная ставка, максимальная сумма
important_criteria = ['Средняя ставка по кредиту %', 'min ставка %', 'max сумма, руб.']

# Упорядочивание банков по лексикографическому порядку
# Сортировка с учетом порядка важности критериев (по убыванию для max сумма)
data_sorted = data.sort_values(by=important_criteria, ascending=[True, True, False])

# Шаг 2: Построить Парето-оптимальный фронт по трем критериям
def is_dominated(row, df, criteria):
    for _, other in df.iterrows():
        if all(other[crit] <= row[crit] for crit in criteria) and any(other[crit] < row[crit] for crit in criteria):
            return True
    return False

criteria_for_pareto = important_criteria
pareto_front = data[~data.apply(is_dominated, df=data, criteria=criteria_for_pareto, axis=1)].copy()

# Шаг 3: Проверить, входит ли лучший банк в Парето-оптимальный фронт
best_bank = data_sorted.iloc[0]
is_best_in_pareto = best_bank["БАНК"] in pareto_front["БАНК"].values

# Динамическая визуализация всех банков и Парето-оптимального фронта на 3D-графике
fig = go.Figure()

# Все банки
fig.add_trace(go.Scatter3d(
    x=data[important_criteria[0]],
    y=data[important_criteria[1]],
    z=data[important_criteria[2]],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.5),
    name='Все банки'
))

# Парето-оптимальный фронт
fig.add_trace(go.Scatter3d(
    x=pareto_front[important_criteria[0]],
    y=pareto_front[important_criteria[1]],
    z=pareto_front[important_criteria[2]],
    mode='markers+lines',
    marker=dict(size=7, color='red', opacity=0.8),
    line=dict(color='red', width=2),
    name='Парето-оптимальный фронт'
))

fig.update_layout(
    title='Парето-оптимальный фронт и все банки (3D)',
    scene=dict(
        xaxis_title=important_criteria[0],
        yaxis_title=important_criteria[1],
        zaxis_title=important_criteria[2]
    )
)
fig.show()

# Вывод результатов
print("Лексикографически лучший банк:", best_bank["БАНК"])
print("Входит ли в Парето-оптимальный фронт:", is_best_in_pareto)
print("Парето-оптимальный фронт:")
print(pareto_front[important_criteria])

Лексикографически лучший банк: DeltaCredit
Входит ли в Парето-оптимальный фронт: True
Парето-оптимальный фронт:
    Средняя ставка по кредиту %  min ставка %  max сумма, руб.
4                       10.7500          9.50             15.0
5                        5.5250          4.00            500.0
6                        4.5421          8.00            500.0
9                       10.2500          9.00             75.0
14                       9.6250          8.75            300.0
